# **Twitter sentiment Analysis**

## part 1 : practical example using SentiWordnet

#**Importing Required Libraries**

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

sentence = "One of the best movie of all time. Period."

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hinnovis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\Hinnovis\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hinnovis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hinnovis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hinnovis\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#**Text PreProcessing**

#### (1) Removing Punctuations

In [ ]:
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

for x in sentence: 
  if x in punctuations: 
    sentence = sentence.replace(x, "")

print(sentence)

One of the best movie of all time Period


#### (2) Change Case + Tokenization

In [ ]:
Tokens = nltk.word_tokenize(sentence.lower())
print(Tokens)

['one', 'of', 'the', 'best', 'movie', 'of', 'all', 'time', 'period']


#### (3) Removing Stop Words - a, an, the, are, is etc.

In [ ]:
stop_words = set(stopwords.words('english'))
clean_Tokens = [word for word in Tokens if word not in stop_words]
print(clean_Tokens)

['one', 'best', 'movie', 'time', 'period']


#### (4) Lemmatization

In [ ]:
lemma = [lemmatizer.lemmatize(word) for word in clean_Tokens]

print(lemma)

['one', 'best', 'movie', 'time', 'period']


#### (5) POS Tagging

A part-of-speech tagger, or POS-tagger, processes a sequence of words, and attaches a part of speech tag to each word (don't forget to import nltk)

In [ ]:
lemma

['one', 'best', 'movie', 'time', 'period']

In [ ]:
help(nltk.pos_tag)

Help on function pos_tag in module nltk.tag:

pos_tag(tokens, tagset=None, lang='eng')
    Use NLTK's currently recommended part of speech tagger to
    tag the given list of tokens.
    
        >>> from nltk.tag import pos_tag
        >>> from nltk.tokenize import word_tokenize
        >>> pos_tag(word_tokenize("John's big idea isn't all that bad.")) # doctest: +NORMALIZE_WHITESPACE
        [('John', 'NNP'), ("'s", 'POS'), ('big', 'JJ'), ('idea', 'NN'), ('is', 'VBZ'),
        ("n't", 'RB'), ('all', 'PDT'), ('that', 'DT'), ('bad', 'JJ'), ('.', '.')]
        >>> pos_tag(word_tokenize("John's big idea isn't all that bad."), tagset='universal') # doctest: +NORMALIZE_WHITESPACE
        [('John', 'NOUN'), ("'s", 'PRT'), ('big', 'ADJ'), ('idea', 'NOUN'), ('is', 'VERB'),
        ("n't", 'ADV'), ('all', 'DET'), ('that', 'DET'), ('bad', 'ADJ'), ('.', '.')]
    
    NB. Use `pos_tag_sents()` for efficient tagging of more than one sentence.
    
    :param tokens: Sequence of tokens to be tagged

In [ ]:
pos_val = nltk.pos_tag(lemma)
print(pos_val)

pos=neg=obj=count=0

[('one', 'CD'), ('best', 'JJS'), ('movie', 'NN'), ('time', 'NN'), ('period', 'NN')]


CC, a coordinating conjunction;

RB, or adverbs;  

IN, a preposition; 

NN, a noun; 

JJ, an adjective.

The most popular tag set is Penn Treebank tagset. Most of the already trained taggers for English are trained on this tag set. To view the complete list, follow this link--> https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [ ]:
# We can get more details about any POS tag using help funciton of NLTK as follows.
nltk.download('tagsets')
nltk.help.upenn_tagset("JJS$")

JJS: adjective, superlative
    calmest cheapest choicest classiest cleanest clearest closest commonest
    corniest costliest crassest creepiest crudest cutest darkest deadliest
    dearest deepest densest dinkiest ...


[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Hinnovis\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [ ]:
nltk.download('universal_tagset')
pos_val2 = nltk.pos_tag(lemma, tagset='universal')
print(pos_val2)

[('one', 'NUM'), ('best', 'ADJ'), ('movie', 'NOUN'), ('time', 'NOUN'), ('period', 'NOUN')]


[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Hinnovis\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


#**Functions for Sentiment Scoring**

In [ ]:
# Convert between the PennTreebank tags to simple Wordnet tags
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

you can find the whol peeTreebank here https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [ ]:
# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    #lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    #if not lemma:
     #   return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

In [ ]:
get_sentiment('one', 'NUM')

['one.n.01', 0.0, 0.0, 1.0]

In [ ]:
senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
print(senti_val)

[[], ['best.a.01', 0.75, 0.0, 0.25], ['movie.n.01', 0.0, 0.0, 1.0], ['time.n.01', 0.0, 0.0, 1.0], ['time_period.n.01', 0.125, 0.125, 0.75]]


#**Aggregating Scores**

In [ ]:
for i in range(len(senti_val)):
  try:
    pos = pos + senti_val[i][1]
    neg = neg + senti_val[i][2]
    obj = obj + senti_val[i][3]
  
  except:
    continue

In [ ]:
print("Positive weight : {0} ".format(pos))
print("Negative weight : {0} ".format(neg))
print("Sentiment of the statement is {0} ".format(pos - neg))

Positive weight : 0.875 
Negative weight : 0.125 
Sentiment of the statement is 0.75 


In [ ]:
## part 2: twitter sentiment Analysis

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
Tweet = pd.read_excel("US_Airline_Tweets.xlsx") # Weather_Final.csv # Airline_Final.csv

In [ ]:
data=Tweet['Tweets']
data

0                    @VirginAmerica What @dhepburn said.
1      @VirginAmerica plus you've added commercials t...
2      @VirginAmerica I didn't today... Must mean I n...
3      @VirginAmerica it's really aggressive to blast...
4      @VirginAmerica and it's a really big bad thing...
                             ...                        
995    @united #UnitedAirlines Pls Fix #AspenBaggageF...
996    @united Read my bio. See who I work with. I ha...
997    @united Does customer care have email or a pho...
998    Thank you â€œ@united: @TRUU_Tall I can certain...
999    @united In the process of recovering their car...
Name: Tweets, Length: 1000, dtype: object

In [ ]:
import re 
#preprocessing
def tweet_preprocess(raw_tweet):
    
    raw_tweet = re.sub("@\w+","",raw_tweet).strip()
    raw_tweet = re.sub("http\S+","",raw_tweet).strip()
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    lemma = [lemmatizer.lemmatize(word) for word in meaningful_words]
    return lemma#( " ".join( meaningful_words )) 

In [ ]:
new_data=[]
for raw in data:
    raw=tweet_preprocess(raw)
    new_data.append(raw)
    print(raw)

['said']
['plus', 'added', 'commercial', 'experience', 'tacky']
['today', 'must', 'mean', 'need', 'take', 'another', 'trip']
['really', 'aggressive', 'blast', 'obnoxious', 'entertainment', 'guest', 'face', 'amp', 'little', 'recourse']
['really', 'big', 'bad', 'thing']
['seriously', 'would', 'pay', 'flight', 'seat', 'playing', 'really', 'bad', 'thing', 'flying', 'va']
['yes', 'nearly', 'every', 'time', 'fly', 'vx', 'ear', 'worm', 'go', 'away']
['really', 'missed', 'prime', 'opportunity', 'men', 'without', 'hat', 'parody']
['well']
['amazing', 'arrived', 'hour', 'early', 'good']
['know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teen']
['lt', 'pretty', 'graphic', 'much', 'better', 'minimal', 'iconography']
['great', 'deal', 'already', 'thinking', 'nd', 'trip', 'amp', 'even', 'gone', 'st', 'trip', 'yet', 'p']
['flying', 'fabulous', 'seductive', 'sky', 'u', 'take', 'stress', 'away', 'travel']
['thanks']
['sfo', 'pdx', 'schedule', 'still', 'mia']
['excited', 'first', 'cros

['come', 'back', 'phl', 'already', 'need', 'take', 'u', 'horrible', 'cold', 'pleasecomeback']
['concerned', 'fly', 'plane', 'need', 'delayed', 'due', 'tech', 'stop']
['best', 'airline', 'flown', 'easy', 'change', 'reservation', 'helpful', 'representative', 'amp', 'comfortable', 'flying', 'experience']
['another', 'rep', 'kicked', 'butt', 'naelah', 'represents', 'team', 'beautifully', 'thank']
['beautiful', 'front', 'end', 'design', 'right', 'cool', 'still', 'book', 'ticket', 'b', 'c', 'back', 'end', 'secure']
['love', 'team', 'running', 'gate', 'e', 'la', 'tonight', 'waited', 'delayed', 'flight', 'kept', 'thing', 'entertaining']
['use', 'another', 'browser', 'amp', 'brand', 'reputation', 'built', 'tech', 'response', 'cross', 'browser', 'compatible', 'website']
['flight', 'flight', 'booking', 'problem', 'site', 'totally', 'folk', 'problem']
['like', 'customer', 'service', 'min', 'delay', 'connecting', 'passenger', 'seems', 'long', 'va']
['thanks', 'outstanding', 'nyc', 'jfk', 'crew', 'm

['hot', 'female', 'pilot', 'sweet', 'dca', 'sfo']
['guy', 'going', 'start', 'flying', 'paris', 'welcome']
['policy', 'flying', 'surgery', 'still', 'waiting', 'answer', 'tell', 'doctor']
['already', 'contacted', 'central', 'baggage', 'amp', 'sent', 'dm', 'charging', 'dress', 'lost', 'suitcase', 'help']
['thank']
['anything', 'going', 'website', 'getting', 'lot', 'error', 'past', 'minute']
['think', 'left', 'something', 'plane', 'yesterday', 'call', 'ah']
['need', 'follow', 'back', 'order', 'dm']
['need', 'start', 'flying']
['told', 'check', 'airline', 'regulation', 'first', 'contacted']
['checked', 'flight', 'sfo', 'lax', 'amp', 'told', 'atlantic', 'flying', 'club', 'gold', 'get', 'benefit']
['husband', 'ordered', 'three', 'drink', 'via', 'screen', 'never', 'came', 'awesome']
['soooo', 'guy', 'going', 'leave', 'seatbelt', 'light', 'flight', 'barely', 'call', 'turbulence']
['policy', 'flying', 'surgery']
['post', 'second', 'consecutive', 'full', 'year', 'net', 'profit']
['funny', 'story'

['virgin', 'america', 'fly', 'direct', 'seattle', 'nyc', 'boston']
['classy']
['sure', 'lot', 'jfk', 'lhr', 'flight', 'go', 'lot', 'faster', 'mph', 'strong', 'tailwind']
['minus', 'minute', 'elevate', 'silver', 'upgrade', 'window', 'open', 'freeneversucks']
['save', 'tomorrow']
['taking', 'year', 'fly', 'home', 'dallas', 'get', 'shit', 'together']
['club', 'virgin', 'bumping', 'new', 'york']
['would', 'rough', 'trip', 'luckily', 'virginamerica', 'flight', 'weather']
['thestarter']
['please', 'come', 'minneapolis', 'st', 'paul']
['yes', 'fyi', 'bff']
['another', 'perfect', 'flight', 'come', 'plane', 'sun', 'visor', 'stay', 'carrier', 'make', 'raise']
['nice', 'rt', 'man', 'steel', 'might', 'faster', 'wifi', 'saying', 'sciencebehindtheexperience']
['love', 'dancing', 'little', 'richard', 'cool', 'bean']
['use', 'passbook', 'still', 'love', 'though', 'lt', 'use', 'email', 'future']
['thanks', 'free', 'birthday', 'point', 'right']
['application', 'io', 'looking', 'saw', 'virgin', 'mexico',

['finger', 'crossed', 'amp', 'intact', 'disappointed', 'experience', 'actually', 'saying', 'enjoyed', 'using']
['incredibly', 'easy', 'fix', 'would', 'rather', 'stay', 'comfort', 'zone', 'get', 'forced', 'mine']
['july', 'zero', 'excuse', 'date', 'system', 'cause', 'problem', 'like']
['everyone', 'around', 'u', 'sitting', 'friend', 'family', 'member', 'booked', 'ticket', 'october', 'december', 'bought']
['sure', 'talking', 'going', 'nonstop', 'flight', 'sna', 'sfo', 'sfo', 'ewr']
['moving', 'talk', 'service']
['please', 'good', 'weekend']
['asked', 'legitimate', 'question', 'systemwide', 'v', 'localized', 'problem', 'got', 'back', 'unrelate', 'flightd', 'copy', 'paste', 'better']
['time', 'call', 'need', 'ski', 'go', 'airport', 'get', 'faith', 'united']
['see', 'guy', 'amazing', 'one', 'company', 'continually', 'make', 'mistake', 'yet', 'fails', 'held', 'accountable', 'error']
['tourist', 'wonder', 'super', 'tide', 'tide', 'century', 'french', 'u', 'k', 'coast']
['yeah', 'sorry', 'alwa

['hope']
['agent', 'help', 'person', 'front', 'finish', 'walk', 'away', 'guy', 'behind', 'shocked']
['connection', 'ord', 'exact', 'plane', 'checked', 'carry', 'apparently', 'taken', 'amp', 'left', 'chicago', 'pretty', 'ridiculous']
['encouraged', 'cpap', 'tonight']
['yes', 'file', 'claim', 'soon', 'carousel', 'stopped', 'without', 'bag', 'got', 'email', 'found', 'sched', 'delivery']
['idea', 'monitor', 'mileage', 'member', 'travel', 'pattern', 'enough', 'know', 'change', 'job', 'lose', 'status', 'make', 'transition', 'easier']
['delay', 'den', 'cle', 'manually', 'enter', 'baggage', 'tag', 'really', 'worst', 'cust', 'service', 'day', 'ker', 'friendlyskies']
['tell', 'u', 'flight', 'delayed', 'tell', 'u', 'time', 'min', 'spare', 'say', 'delayed', 'unless', 'sure']
['yes', 'lusaka', 'zambia', 'guess', 'bag', 'never', 'made', 'plane', 'iad']
['flight', 'voucher']
['late', 'flight', 'denver', 'late', 'flight', 'newark', 'let', 'even', 'get', 'disaster', 'checking', 'bag', 'unacceptable']
[

['really', 'someone', 'called', 'sick', 'someone', 'forgot', 'call', 'replacement', 'hour', 'late', 'flight', 'take', 'theworst']
['asked', 'flight', 'attendant', 'typical', 'compensation', 'would', 'amp', 'say', 'might', 'get', 'free', 'tv', 'unacceptable']
['done']
['thank', 'jh', 'appreciate', 'prompt', 'response', 'passenger']
['disappointment', 'today', 'simply', 'put', 'rather', 'unpleased', 'thing', 'currently', 'flight', 'delayed', 'cant', 'wait', 'get', 'home']
['round', 'trip', 'non', 'english', 'speaking', 'friend', 'stranded', 'twice', 'hour', 'time', 'staff', 'member', 'rude']
['found', 'group', 'people', 'actually', 'hate', 'people', 'isi', 'thats', 'right', 'united', 'airline']
['nope', 'walked', 'amp', 'orf', 'terminal', 'amp', 'united', 'agent', 'found', 'showed', 'last', 'minute', 'board', 'u', 'many', 'panicked', 'people']
['thank']
['time', 'finally', 'get', 'dallas', 'could', 'driven', 'le', 'frustration', 'cheaper']
['trying', 'get', 'final', 'destination', 'need'

In [ ]:
new_data

[['said'],
 ['plus', 'added', 'commercial', 'experience', 'tacky'],
 ['today', 'must', 'mean', 'need', 'take', 'another', 'trip'],
 ['really',
  'aggressive',
  'blast',
  'obnoxious',
  'entertainment',
  'guest',
  'face',
  'amp',
  'little',
  'recourse'],
 ['really', 'big', 'bad', 'thing'],
 ['seriously',
  'would',
  'pay',
  'flight',
  'seat',
  'playing',
  'really',
  'bad',
  'thing',
  'flying',
  'va'],
 ['yes', 'nearly', 'every', 'time', 'fly', 'vx', 'ear', 'worm', 'go', 'away'],
 ['really',
  'missed',
  'prime',
  'opportunity',
  'men',
  'without',
  'hat',
  'parody'],
 ['well'],
 ['amazing', 'arrived', 'hour', 'early', 'good'],
 ['know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teen'],
 ['lt', 'pretty', 'graphic', 'much', 'better', 'minimal', 'iconography'],
 ['great',
  'deal',
  'already',
  'thinking',
  'nd',
  'trip',
  'amp',
  'even',
  'gone',
  'st',
  'trip',
  'yet',
  'p'],
 ['flying',
  'fabulous',
  'seductive',
  'sky',
  'u',
  'ta

## To do : for each tweet get it sentiment using sentiwordnet
NB : add you name and ubmit your work under Lab4_submission 

In [ ]:
for i in range(0,len(new_data)):
    print("***********the tweet number {} *********************************".format(i+1))
    pos_val = nltk.pos_tag(new_data[i])
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    
    for j in range(len(senti_val)):
        try:
            pos = pos + senti_val[j][1]
            neg = neg + senti_val[j][2]
            obj = obj + senti_val[j][3]
        except:
         continue
        print("Positive weight : {0} ".format(pos))
        print("Negative weight : {0} ".format(neg))
        print("Sentiment of the statement is {0} ".format(pos - neg))


***********the tweet number 1 *********************************
***********the tweet number 2 *********************************
Positive weight : 693.25 
Negative weight : 620.0 
Sentiment of the statement is 73.25 
Positive weight : 693.25 
Negative weight : 620.0 
Sentiment of the statement is 73.25 
***********the tweet number 3 *********************************
Positive weight : 693.375 
Negative weight : 620.0 
Sentiment of the statement is 73.375 
Positive weight : 693.375 
Negative weight : 620.0 
Sentiment of the statement is 73.375 
***********the tweet number 4 *********************************
Positive weight : 694.0 
Negative weight : 620.0 
Sentiment of the statement is 74.0 
Positive weight : 694.5 
Negative weight : 620.0 
Sentiment of the statement is 74.5 
Positive weight : 694.5 
Negative weight : 620.125 
Sentiment of the statement is 74.375 
Positive weight : 694.5 
Negative weight : 620.75 
Sentiment of the statement is 73.75 
Positive weight : 694.5 
Negative weig

Positive weight : 711.5 
Negative weight : 629.625 
Sentiment of the statement is 81.875 
Positive weight : 711.5 
Negative weight : 629.625 
Sentiment of the statement is 81.875 
Positive weight : 712.0 
Negative weight : 629.625 
Sentiment of the statement is 82.375 
***********the tweet number 39 *********************************
Positive weight : 712.0 
Negative weight : 629.625 
Sentiment of the statement is 82.375 
Positive weight : 712.0 
Negative weight : 629.625 
Sentiment of the statement is 82.375 
Positive weight : 712.0 
Negative weight : 629.625 
Sentiment of the statement is 82.375 
***********the tweet number 40 *********************************
Positive weight : 712.0 
Negative weight : 629.625 
Sentiment of the statement is 82.375 
Positive weight : 712.0 
Negative weight : 629.625 
Sentiment of the statement is 82.375 
***********the tweet number 41 *********************************
Positive weight : 712.0 
Negative weight : 629.625 
Sentiment of the statement is 82.

Positive weight : 719.625 
Negative weight : 634.625 
Sentiment of the statement is 85.0 
Positive weight : 719.625 
Negative weight : 634.625 
Sentiment of the statement is 85.0 
***********the tweet number 71 *********************************
Positive weight : 719.75 
Negative weight : 634.75 
Sentiment of the statement is 85.0 
Positive weight : 719.75 
Negative weight : 634.75 
Sentiment of the statement is 85.0 
Positive weight : 719.75 
Negative weight : 634.75 
Sentiment of the statement is 85.0 
Positive weight : 719.75 
Negative weight : 634.875 
Sentiment of the statement is 84.875 
Positive weight : 720.125 
Negative weight : 635.125 
Sentiment of the statement is 85.0 
Positive weight : 720.125 
Negative weight : 635.125 
Sentiment of the statement is 85.0 
Positive weight : 720.125 
Negative weight : 635.125 
Sentiment of the statement is 85.0 
Positive weight : 720.125 
Negative weight : 635.125 
Sentiment of the statement is 85.0 
Positive weight : 720.125 
Negative weig

Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
***********the tweet number 93 *********************************
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative weight : 639.5 
Sentiment of the statement is 86.875 
Positive weight : 726.375 
Negative

Positive weight : 733.625 
Negative weight : 645.0 
Sentiment of the statement is 88.625 
Positive weight : 733.625 
Negative weight : 645.0 
Sentiment of the statement is 88.625 
Positive weight : 734.25 
Negative weight : 645.25 
Sentiment of the statement is 89.0 
Positive weight : 734.25 
Negative weight : 645.25 
Sentiment of the statement is 89.0 
Positive weight : 734.25 
Negative weight : 645.25 
Sentiment of the statement is 89.0 
Positive weight : 734.5 
Negative weight : 645.25 
Sentiment of the statement is 89.25 
Positive weight : 734.5 
Negative weight : 645.25 
Sentiment of the statement is 89.25 
Positive weight : 734.5 
Negative weight : 645.25 
Sentiment of the statement is 89.25 
Positive weight : 734.5 
Negative weight : 646.0 
Sentiment of the statement is 88.5 
Positive weight : 734.5 
Negative weight : 646.0 
Sentiment of the statement is 88.5 
***********the tweet number 118 *********************************
Positive weight : 734.5 
Negative weight : 646.0 
Sent

...

***********the tweet number 865 *********************************
Positive weight : 992.875 
Negative weight : 873.5 
Sentiment of the statement is 119.375 
Positive weight : 992.875 
Negative weight : 873.5 
Sentiment of the statement is 119.375 
***********the tweet number 866 *********************************
Positive weight : 993.0 
Negative weight : 873.5 
Sentiment of the statement is 119.5 
Positive weight : 993.0 
Negative weight : 873.5 
Sentiment of the statement is 119.5 
Positive weight : 993.0 
Negative weight : 873.5 
Sentiment of the statement is 119.5 
Positive weight : 993.0 
Negative weight : 873.5 
Sentiment of the statement is 119.5 
Positive weight : 993.0 
Negative weight : 873.5 
Sentiment of the statement is 119.5 
Positive weight : 993.0 
Negative weight : 873.5 
Sentiment of the statement is 119.5 
***********the tweet number 867 *********************************
Positive weight : 993.0 
Negative weight : 873.5 
Sentiment of the statement is 119.5 
Positive we

Sentiment of the statement is 120.875 
Positive weight : 1004.125 
Negative weight : 882.5 
Sentiment of the statement is 121.625 
Positive weight : 1004.125 
Negative weight : 882.5 
Sentiment of the statement is 121.625 
***********the tweet number 897 *********************************
Positive weight : 1004.125 
Negative weight : 882.5 
Sentiment of the statement is 121.625 
***********the tweet number 898 *********************************
Positive weight : 1004.125 
Negative weight : 882.5 
Sentiment of the statement is 121.625 
Positive weight : 1004.125 
Negative weight : 882.5 
Sentiment of the statement is 121.625 
Positive weight : 1004.625 
Negative weight : 882.625 
Sentiment of the statement is 122.0 
***********the tweet number 899 *********************************
Positive weight : 1004.625 
Negative weight : 882.625 
Sentiment of the statement is 122.0 
***********the tweet number 900 *********************************
Positive weight : 1005.375 
Negative weight : 882.625

Positive weight : 1010.125 
Negative weight : 887.875 
Sentiment of the statement is 122.25 
Positive weight : 1010.25 
Negative weight : 888.375 
Sentiment of the statement is 121.875 
Positive weight : 1010.25 
Negative weight : 888.375 
Sentiment of the statement is 121.875 
Positive weight : 1010.25 
Negative weight : 888.875 
Sentiment of the statement is 121.375 
Positive weight : 1010.25 
Negative weight : 889.5 
Sentiment of the statement is 120.75 
Positive weight : 1010.75 
Negative weight : 889.5 
Sentiment of the statement is 121.25 
Positive weight : 1010.875 
Negative weight : 889.5 
Sentiment of the statement is 121.375 
Positive weight : 1011.25 
Negative weight : 889.5 
Sentiment of the statement is 121.75 
Positive weight : 1011.25 
Negative weight : 889.5 
Sentiment of the statement is 121.75 
Positive weight : 1011.25 
Negative weight : 889.5 
Sentiment of the statement is 121.75 
***********the tweet number 925 *********************************
Positive weight : 10

Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
***********the tweet number 957 *********************************
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
Positive weight : 1021.75 
Negative weight : 906.875 
Sentiment of the statement is 114.875 
****

Positive weight : 1032.875 
Negative weight : 922.0 
Sentiment of the statement is 110.875 
Positive weight : 1032.875 
Negative weight : 922.0 
Sentiment of the statement is 110.875 
Positive weight : 1033.125 
Negative weight : 922.25 
Sentiment of the statement is 110.875 
Positive weight : 1033.625 
Negative weight : 922.25 
Sentiment of the statement is 111.375 
Positive weight : 1033.625 
Negative weight : 922.5 
Sentiment of the statement is 111.125 
Positive weight : 1033.625 
Negative weight : 922.5 
Sentiment of the statement is 111.125 
Positive weight : 1033.625 
Negative weight : 922.5 
Sentiment of the statement is 111.125 
Positive weight : 1033.625 
Negative weight : 922.5 
Sentiment of the statement is 111.125 
Positive weight : 1033.625 
Negative weight : 922.5 
Sentiment of the statement is 111.125 
Positive weight : 1033.625 
Negative weight : 922.5 
Sentiment of the statement is 111.125 
***********the tweet number 982 *********************************
***********t